In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#### Load Model

In [2]:
net = cv2.dnn.readNetFromDarknet('yolov3-spp.cfg','yolov3-spp.weights')

In [3]:
classes = []

with open ('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


### Pose Detection

In [5]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()  # Needed before execution!

In [6]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

In [7]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [8]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 2)

# Make Detections

In [9]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    ## Object detection
    
    img = frame.copy()
    img_od = cv2.resize(frame,(1280,720))
    hight,width,_ = img_od.shape
    blob = cv2.dnn.blobFromImage(img_od, 1/255,(416,416),(0,0,0),swapRB = True,crop= False)

    net.setInput(blob)

    output_layers_name = net.getUnconnectedOutLayersNames()

    layerOutputs = net.forward(output_layers_name)

    boxes =[]
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            score = detection[5:]
            class_id = np.argmax(score)
            confidence = score[class_id]
            if confidence > 0.7:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * hight)
                w = int(detection[2] * width)
                h = int(detection[3]* hight)
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                boxes.append([x,y,w,h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,.5,.4)

    boxes =[]
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            score = detection[5:]
            class_id = np.argmax(score)
            confidence = score[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * hight)
                w = int(detection[2] * width)
                h = int(detection[3]* hight)

                x = int(center_x - w/2)
                y = int(center_y - h/2)



                boxes.append([x,y,w,h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,.8,.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0,255,size =(len(boxes),3))
    if  len(indexes)>0:
        for i in indexes.flatten():
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(img_od,(x,y),(x+w,y+h),color,2)
            cv2.putText(img_od,label + " " + confidence, (x,y+400),font,2,color,2)

    
    ##Pose estimation
    
    ## Reshape image ______ 
    ##A frame of video or an image, represented as an float32 tensor of shape: 192x192x3. Channels order: RGB with values in [0, 255].
    img_pd = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img_pd, dtype=tf.float32)
    
    ## setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    ## Make predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    ##Rendering
    draw_connections(img_od, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(img_od, keypoints_with_scores, 0.4)
    
    
    cv2.imshow('Object_Pose_Detection', img_od)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[[[[0.684011   0.7092706  0.55132043]
   [0.5479117  0.85856855 0.5188441 ]
   [0.53487295 0.5838332  0.5102696 ]
   [0.5530559  0.9908344  0.2983431 ]
   [0.51449656 0.4888675  0.58828247]
   [0.8236388  0.9921884  0.11924224]
   [0.7463435  0.33181673 0.5842687 ]
   [0.9659826  0.95266145 0.02902581]
   [0.85569876 0.02061893 0.06342664]
   [0.8686194  0.91167617 0.03054725]
   [0.86973256 0.45131004 0.00994825]
   [0.6262412  1.0000858  0.00884227]
   [0.7599155  0.01408749 0.0619489 ]
   [0.8326699  0.01423273 0.05770183]
   [0.8341508  0.01112202 0.07291913]
   [0.85122555 0.01804592 0.08481807]
   [0.8518729  0.02085043 0.10188704]]]]
[[[[0.6729104  0.7230948  0.41746947]
   [0.5484253  0.876151   0.48736048]
   [0.5309315  0.6094608  0.5187233 ]
   [0.57975703 0.99466795 0.22894472]
   [0.529093   0.51262724 0.5080117 ]
   [0.8430647  0.9925608  0.08971278]
   [0.7525513  0.34080872 0.5788757 ]
   [0.873481   0.9845197  0.03123105]
   [0.8595985  0.21913323 0.07402732]
   [0.783

[[[[0.448277   0.51210904 0.7343254 ]
   [0.41222733 0.570052   0.51694024]
   [0.4153534  0.45956215 0.59076256]
   [0.48701608 0.6407763  0.70954776]
   [0.49406636 0.3934915  0.57640946]
   [0.69557333 0.78743196 0.81887096]
   [0.7040981  0.2785318  0.866068  ]
   [0.7182161  0.82375145 0.01082654]
   [0.7124902  0.10872835 0.09655893]
   [0.7219984  0.10465693 0.06222272]
   [0.72257966 0.10038249 0.07914764]
   [0.40595755 0.72600526 0.01632861]
   [0.73839223 0.12834018 0.02444653]
   [0.7111098  0.8233029  0.05310183]
   [0.6960656  0.28686446 0.08878319]
   [0.52734923 0.6567971  0.0116144 ]
   [0.739202   0.10112312 0.0630337 ]]]]
[[[[0.47239575 0.52284    0.5881804 ]
   [0.43248042 0.5793571  0.60921526]
   [0.43693325 0.46520624 0.5637335 ]
   [0.49652094 0.64951974 0.46304563]
   [0.50861865 0.39992207 0.7239574 ]
   [0.69739974 0.78424156 0.77315974]
   [0.7013109  0.28878695 0.8351614 ]
   [0.7869364  0.85306    0.00918456]
   [0.71391433 0.10346875 0.07826499]
   [0.718

[[[[0.48083866 0.5563886  0.57453537]
   [0.42934573 0.6155932  0.69556296]
   [0.43262136 0.4951571  0.58780265]
   [0.48286527 0.6806847  0.7576361 ]
   [0.49109244 0.4279327  0.76528335]
   [0.69947076 0.8286989  0.7932734 ]
   [0.7078341  0.35962564 0.63419795]
   [0.8357312  0.97214836 0.05593296]
   [0.8677627  0.22661385 0.21046476]
   [0.78814816 0.9814362  0.04178499]
   [0.76965445 0.16558975 0.26455817]
   [0.4123209  0.3493143  0.02287731]
   [0.3911758  0.3659261  0.03549737]
   [0.81119764 0.9761739  0.02702282]
   [0.6820039  0.37438008 0.01846037]
   [0.5352263  0.8591926  0.02169406]
   [0.7843616  0.15064299 0.03146653]]]]
[[[[0.48256406 0.52312666 0.591885  ]
   [0.43471214 0.57399344 0.6227739 ]
   [0.44045585 0.46418756 0.66973686]
   [0.4821388  0.6446694  0.43199557]
   [0.4868868  0.39896214 0.726609  ]
   [0.69214654 0.82258165 0.53044856]
   [0.72073436 0.332624   0.7236439 ]
   [0.8099179  0.97414637 0.0724142 ]
   [0.857906   0.28481132 0.06094977]
   [0.595

[[[[0.4643998  0.4946191  0.64490294]
   [0.41983354 0.5413327  0.6131742 ]
   [0.42525768 0.42786407 0.7576652 ]
   [0.4703881  0.5941582  0.7226193 ]
   [0.49457926 0.35155436 0.46052113]
   [0.67328155 0.7241854  0.7744154 ]
   [0.7197404  0.2530585  0.673513  ]
   [0.8862981  0.89320314 0.3535228 ]
   [0.84965765 0.0987772  0.02731648]
   [0.76222336 0.9626279  0.12127183]
   [0.61912024 0.22244093 0.01580598]
   [0.7940282  0.94714355 0.05391171]
   [0.7906773  0.94521207 0.05474321]
   [0.575639   0.8299211  0.05823299]
   [0.70016694 0.26674336 0.07538311]
   [0.7615781  0.9674127  0.06557661]
   [0.76344675 0.961588   0.04825437]]]]
[[[[0.44990563 0.48564085 0.72194123]
   [0.41214117 0.5302237  0.55512285]
   [0.4109107  0.42288935 0.67771673]
   [0.47584125 0.57939756 0.6178621 ]
   [0.4831997  0.33490485 0.6546171 ]
   [0.6541959  0.7131457  0.65562433]
   [0.7268193  0.23468201 0.78488487]
   [0.83002466 0.7642225  0.18336561]
   [0.79378664 0.03016242 0.02256525]
   [0.550

[[[[0.463987   0.4859937  0.69216186]
   [0.41827792 0.52956796 0.73724025]
   [0.4170887  0.415726   0.5708504 ]
   [0.47374853 0.58230126 0.6282306 ]
   [0.48481795 0.3279139  0.82828337]
   [0.6838149  0.725876   0.86831325]
   [0.72731113 0.21626976 0.6699238 ]
   [0.95152044 0.9074931  0.02848465]
   [0.72717226 0.10554525 0.10438461]
   [0.43893152 0.6800927  0.0725318 ]
   [0.7216887  0.0919981  0.1015591 ]
   [0.7576086  0.11856373 0.03334672]
   [0.77589315 0.13865346 0.05003588]
   [0.67729795 0.75821507 0.076084  ]
   [0.730782   0.10389687 0.2087223 ]
   [0.730991   0.09988359 0.08340798]
   [0.7310774  0.09713774 0.08084104]]]]
[[[[0.46761864 0.48360652 0.70634353]
   [0.42182156 0.53014827 0.7676308 ]
   [0.41746634 0.4162993  0.5851226 ]
   [0.4769643  0.5915091  0.5473793 ]
   [0.4816601  0.33096087 0.7544301 ]
   [0.68741846 0.71877944 0.8646    ]
   [0.7081377  0.22222397 0.80006576]
   [0.8659383  0.9671682  0.04265394]
   [0.8502976  0.09689503 0.1315654 ]
   [0.404

[[[[0.55634415 0.39372358 0.41133812]
   [0.5009581  0.37971148 0.3812821 ]
   [0.50342816 0.3242008  0.33860123]
   [0.53561866 0.34704888 0.09327904]
   [0.67263293 0.07325101 0.40732747]
   [0.7123868  0.4474817  0.3472321 ]
   [0.86026305 0.04084619 0.11671411]
   [0.86288357 0.56526995 0.17583856]
   [0.8275975  0.0223724  0.02722466]
   [0.68519366 0.4426349  0.0557586 ]
   [0.6762253  0.09267118 0.04117264]
   [0.8837943  0.43834445 0.0824196 ]
   [0.5865643  0.367862   0.02069212]
   [0.46132752 0.8982562  0.07464769]
   [0.64558035 0.41272137 0.02814351]
   [0.7489285  0.95878184 0.08076804]
   [0.7530412  0.9580004  0.10981937]]]]
[[[[0.6513364  0.35565218 0.5200844 ]
   [0.5525222  0.38003236 0.44735205]
   [0.60480225 0.23738465 0.31068173]
   [0.5532316  0.3965586  0.39952007]
   [0.6870583  0.09517401 0.54674256]
   [0.7519287  0.5294646  0.49562672]
   [0.8496051  0.02148041 0.18632737]
   [0.8441276  0.60573655 0.02947024]
   [0.9567346  0.0155189  0.02752561]
   [0.777

In [ ]:
# ! cd '/content/drive/My Drive/Colab Notebooks/YOLO_object_detection'